<a href="https://colab.research.google.com/github/evanoinogueira/Triagem_Veterinaria/blob/main/Triagem_Veterin%C3%A1ria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install -q google-generativeai

import google.generativeai as genai
from IPython.display import Markdown, display
import os


try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key configurada com sucesso a partir dos Secrets do Colab.")
except ImportError:

    print("Secrets do Colab não encontrados. Por favor, insira sua API Key manualmente.")
    print("Você pode obter uma API Key em: https://aistudio.google.com/app/apikey")
    GOOGLE_API_KEY = input("Coloque sua Google API Key")
    if GOOGLE_API_KEY:
        genai.configure(api_key=GOOGLE_API_KEY)
        print("API Key configurada manualmente.")
    else:
        print("API Key não fornecida. O chatbot não funcionará.")


# --- Definição do Modelo e Configurações ---


model = genai.GenerativeModel('gemini-1.5-flash')


generation_config = {
    "temperature": 0.7,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 2048,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]
# --- Lógica do Chatbot de Triagem Veterinária ---

def exibir_markdown(texto):
    """Função auxiliar para exibir texto formatado em Markdown."""
    display(Markdown(texto))

def iniciar_chat_triagem():
    exibir_markdown("---")
    exibir_markdown("🐾 **Assistente Virtual de Triagem Veterinária** 🐾")
    exibir_markdown("Olá! Sou um assistente virtual para uma triagem inicial de sintomas do seu pet.")
    exibir_markdown("**EU NÃO SUBSTITUO UM VETERINÁRIO.** As informações aqui não são diagnósticos.")
    exibir_markdown("Ao final da conversa, **SEMPRE recomendo que você procure um veterinário.**")
    exibir_markdown("---")



    # Usando o método start_chat para conversas multi-turn
    chat = model.start_chat(history=[
        {
            "role": "user",
            "parts": [
                """Você é um assistente virtual de IA para triagem inicial de sintomas em animais de estimação.
                Seu objetivo é fazer algumas perguntas básicas para entender a situação do pet e, ao final, SEMPRE recomendar que o usuário procure um veterinário para um diagnóstico e tratamento adequados.
                VOCÊ NÃO DEVE FORNECER DIAGNÓSTICOS OU TRATAMENTOS.
                Seu tom deve ser empático e cuidadoso.

                Pergunte sobre:
                1. Qual o animal (cão, gato, etc.), raça (se souber) e idade aproximada.
                2. Qual o principal sintoma que o preocupa.
                3. Há quanto tempo o sintoma está presente.
                4. Se há outros sintomas (vômito, diarreia, letargia, falta de apetite, tosse, febre, mudanças de comportamento etc.).
                5. O estado geral do animal (alerta, apático, brincalhão como sempre, etc.).
                6. Foi dado algum medicamento?
                7. O que seu animal come?
                8. Tem outros animais com ele?
                9. É castrado?
                10. As vacinas estão em dia?
                11. Quando foi desvermifugado?


                Mantenha a conversa com foco na coleta dessas informações. Após coletar informações suficientes (5 interações),
                forneça uma mensagem final clara enfatizando a necessidade de consultar um veterinário,
                e explicando que a triagem é apenas uma orientação preliminar e não um diagnóstico.
                Não se estenda demais na conversa.
                Inicie a conversa se apresentando brevemente (você já fez isso no prompt acima, então comece com a primeira pergunta).
                Se o tutor responder sangue, dificuldade para respirar, desmaio, convulção, qualquer sintoma em olho, atropelamento,
                envenenamento,  você deverá enfatizar que é EMERGÊNCIA."""
            ]
        },
        {
            "role": "model",
            "parts": [
                "Entendido! Estou aqui para ajudar a fazer uma triagem inicial. Para começar, qual é o seu pet (cão, gato, outro), qual a raça (se souber) e a idade aproximada dele(a)?"
            ]
        }
    ])

    # Exibe a primeira mensagem do modelo

    exibir_markdown(f"**Assistente:** {chat.history[-1].parts[0].text}")


    while True:
        exibir_markdown("---")
        entrada_usuario = input("Você: ")

        if entrada_usuario.lower() in ["sair", "fim", "tchau", "obrigado"]:
            exibir_markdown("**Assistente:** Entendido! Lembre-se que esta foi apenas uma orientação inicial. **Por favor, procure um médico veterinário para avaliar seu pet e obter um diagnóstico e tratamento corretos.** Melhoras para ele(a)!")
            break

        if not GOOGLE_API_KEY:
            exibir_markdown("**Assistente:** Desculpe, a API Key não foi configurada. Não consigo processar seu pedido.")
            break

        try:

            resposta = chat.send_message(entrada_usuario, generation_config=generation_config, safety_settings=safety_settings)


            exibir_markdown(f"**Assistente:** {resposta.text}")


            if "veterinário" in resposta.text.lower() and ("diagnóstico" in resposta.text.lower() or "avaliar" in resposta.text.lower()):
                if len(chat.history) > 6 :
                    exibir_markdown("---")
                    exibir_markdown("**Assistente: **Lembre-se, esta conversa é apenas uma triagem educacional. É crucial levar seu pet a um veterinário para um diagnóstico preciso e tratamento adequado.**")
                    break

        except Exception as e:
            exibir_markdown(f"**Assistente:** Desculpe, ocorreu um erro ao processar sua solicitação: {e}")
            exibir_markdown("Por favor, tente novamente ou, se o problema persistir, **procure um veterinário imediatamente se estiver preocupado com seu pet.**")
            break
    exibir_markdown("---")
    exibir_markdown("Sessão de triagem encerrada.")

# --- Iniciar o Chat ---
if __name__ == "__main__" and GOOGLE_API_KEY:
    iniciar_chat_triagem()
elif not GOOGLE_API_KEY:
    exibir_markdown("**ERRO:** A API Key do Google não foi configurada. O chatbot não pode ser iniciado.")

API Key configurada com sucesso a partir dos Secrets do Colab.


---

🐾 **Assistente Virtual de Triagem Veterinária** 🐾

Olá! Sou um assistente virtual para uma triagem inicial de sintomas do seu pet.

**EU NÃO SUBSTITUO UM VETERINÁRIO.** As informações aqui não são diagnósticos.

...

**Assistente:** Obrigada pelas informações.  Lembre-se que, apesar de parecer que seu cão está bem,  uma tosse persistente por duas semanas precisa ser avaliada por um veterinário.  **É fundamental que você procure um veterinário para um exame completo e diagnóstico preciso.**  Esta conversa foi apenas uma triagem inicial e não substitui a consulta profissional.  Não posso fornecer diagnósticos ou tratamentos.  Por favor, agende uma consulta o mais breve possível.


---

**Assistente: **Lembre-se, esta conversa é apenas uma triagem educacional. É crucial levar seu pet a um veterinário para um diagnóstico preciso e tratamento adequado.**

---

Sessão de triagem encerrada.